In [1]:
import pandas as pd
import Levenshtein
import itertools
from fuzzywuzzy import fuzz, process
import ast
from selenium import webdriver
import time
import webbrowser

In [2]:
recipe_df = pd.read_csv('../data/processed/ingredients_df.csv')

pd.set_option('max_columns', None)

recipe_df.head()

,recipe_name,recipe_url,ingredients_list,ingredient,variable,num_ingredients,category,main_category,tipo_plato,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free
0,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",patatas pequeñas rojas,0,12,ensaladas,principal,['Primeros platos y entrantes'],60.0,252.0,0,0,0,0,0
1,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",foie micuit,0,9,ensaladas,principal,['Primeros platos y entrantes'],25.0,386.0,0,0,0,0,0
2,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",bacalao desalado,0,16,ensaladas,principal,['Primeros platos y entrantes'],35.0,NaN,0,0,0,0,0
3,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",langostinos,0,12,ensaladas,principal,['Primeros platos y entrantes'],25.0,176.0,0,0,0,0,0
4,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",pechugas de pollo,0,16,ensaladas,principal,['Primeros platos y entrantes'],40.0,203.0,0,0,0,0,0


In [3]:
input_list = ['huevos', 'pechiga de pollo', 'lentejas', 'pimiento', 'arroz', 'leche', 'mantequilla',
              'harina', 'sal', 'azúcar', 'aceite', 'zanahorias', 'queso', 'bacon', 'lechuga', 'calabacín',
              'patatas', 'pollo']

In [4]:
def fuzzy_columns(column, input_string):
    ratio = fuzz.WRatio(column, input_string)
    return ratio

In [5]:
for ingred in input_list:
    recipe_df[ingred] = recipe_df.apply(lambda x: fuzzy_columns(x["ingredient"], ingred), axis=1)

In [6]:
recipe_df

,recipe_name,recipe_url,ingredients_list,ingredient,variable,num_ingredients,category,main_category,tipo_plato,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,huevos,pechiga de pollo,lentejas,pimiento,arroz,leche,mantequilla,harina,sal,azúcar,aceite,zanahorias,queso,bacon,lechuga,calabacín,patatas,pollo
0,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",patatas pequeñas rojas,0,12,ensaladas,principal,['Primeros platos y entrantes'],60.0,252.0,0,0,0,0,0,45,32,34,22,36,36,40,30,30,36,30,36,72,18,39,34,90,18
1,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",foie micuit,0,9,ensaladas,principal,['Primeros platos y entrantes'],25.0,386.0,0,0,0,0,0,15,28,21,42,12,25,27,15,0,18,45,19,18,18,39,11,13,18
2,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",bacalao desalado,0,16,ensaladas,principal,['Primeros platos y entrantes'],35.0,NaN,0,0,0,0,0,30,44,34,22,36,19,30,30,90,54,30,36,36,54,26,45,39,36
3,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",langostinos,0,12,ensaladas,principal,['Primeros platos y entrantes'],25.0,176.0,0,0,0,0,0,30,22,42,32,36,18,36,35,30,18,30,48,36,38,26,32,33,25
4,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",pechugas de pollo,0,16,ensaladas,principal,['Primeros platos y entrantes'],40.0,203.0,0,0,0,0,0,45,91,34,24,34,54,32,30,30,18,30,26,36,18,77,24,26,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57309,Pastel de tres colores y tres sabores,https://www.lecturas.com/recetas/pastel-tres-c...,"['mantequilla', 'azúcar', 'huevo', 'aroma de v...",azúcar,30,33,postres y dulces,postre,['Postres'],240.0,NaN,0,0,0,0,0,0,19,18,0,40,20,25,36,30,100,36,40,0,40,33,36,33,0
57310,Tarta de cubos de chocolate con toffee,https://www.lecturas.com/recetas/tarta-cubos-c...,"['mantequilla', 'azúcar', 'huevo', 'harina', '...",chocolate negro fondant,31,33,postres y dulces,postre,['Postres'],360.0,NaN,0,0,0,0,0,30,36,45,34,36,36,31,34,30,36,45,27,36,36,27,45,27,36
57311,Pastel de tres colores y tres sabores,https://www.lecturas.com/recetas/pastel-tres-c...,"['mantequilla', 'azúcar', 'huevo', 'aroma de v...",chocolate negro fondant,31,33,postres y dulces,postre,['Postres'],240.0,NaN,0,0,0,0,0,30,36,45,34,36,36,31,34,30,36,45,27,36,36,27,45,27,36
57312,Tarta de cubos de chocolate con toffee,https://www.lecturas.com/recetas/tarta-cubos-c...,"['mantequilla', 'azúcar', 'huevo', 'harina', '...",cacao en polvo puro,32,33,postres y dulces,postre,['Postres'],360.0,NaN,0,0,0,0,0,30,51,22,22,36,36,24,30,30,54,45,28,36,54,26,45,26,72


In [7]:
input_list

['huevos',
 'pechiga de pollo',
 'lentejas',
 'pimiento',
 'arroz',
 'leche',
 'mantequilla',
 'harina',
 'sal',
 'azúcar',
 'aceite',
 'zanahorias',
 'queso',
 'bacon',
 'lechuga',
 'calabacín',
 'patatas',
 'pollo']

In [31]:
filter_huevos = recipe_df['huevos'] > 86

recipe_df[filter_huevos]["ingredient"].value_counts()

huevo                                          1373
huevos                                          723
huevos de codorniz                               35
huevos grandes                                   10
huevos medianos                                   6
huevos duros                                      2
huevos de chocolate                               1
aceite de oliva suave el peso de los huevos       1
huevos pequeños                                   1
huevos cocido                                     1
huevos cocidos                                    1
azúcar moreno el peso de los huevos               1
harina integral el peso de los huevos             1
Name: ingredient, dtype: int64

In [9]:
filter_pechiga_de_pollo = recipe_df['pechiga de pollo'] > 86
recipe_df[filter_pechiga_de_pollo]["ingredient"].value_counts()

pechugas de pollo    86
pechuga de pollo     14
pollo                 4
Name: ingredient, dtype: int64

In [10]:
filter_lentejas = recipe_df['lentejas'] > 86
recipe_df[filter_lentejas]["ingredient"].value_counts()

lentejas                      23
lentejas cocidas              11
lentejas pardinas cocidas      8
lentejas pardinas              8
lentejas rojas                 2
lentejas rojas lavados         1
paté de lentejas               1
hummus de lentejas rojas       1
lentejas del tipo pardinas     1
Name: ingredient, dtype: int64

In [11]:
filter_pimiento = recipe_df['pimiento'] > 86
recipe_df[filter_pimiento]["ingredient"].value_counts()

pimienta                                                  1484
pimiento rojo                                              318
pimiento verde                                             278
pimientos rojos                                             89
pimientos verdes                                            60
                                                          ... 
pimiento rojo asado 1 tira                                   1
ñora pimiento seco o pimiento choricero                      1
verdura espinacas zanahoria judías vedes pimiento rojo       1
pimiento morrón rojo pequeño                                 1
pimientos del piquillo en tiras                              1
Name: ingredient, Length: 83, dtype: int64

In [37]:
filter_arroz = recipe_df['arroz'] > 86
recipe_df[filter_arroz]["ingredient"].value_counts()

arroz                                      148
arroz redondo                               30
arroz integral                              29
arroz basmati                               29
arroz largo                                 29
arroz bomba                                 19
harina de arroz                             18
vinagre de arroz                            16
leche de arroz                              10
arroz salvaje                                9
arroz arborio                                8
arroz carnaroli                              4
sirope de arroz                              4
arroz inflado de chocolate                   4
fideos de arroz                              4
arroz cocido                                 3
arroz nerone (venere)                        3
morcilla de arroz                            3
oblea de arroz                               3
arroz inflado                                2
pasta de arroz                               2
arroz arborio

In [35]:
filter_leche = recipe_df['leche'] > 86
recipe_df[filter_leche]["ingredient"].value_counts()

leche                                       987
leche condensada                             43
chocolate con leche                          38
leche desnatada                              37
leche de coco                                30
leche de soja                                26
leche evaporada                              25
chocolate con leche fondant                  24
leche de almendras                           12
dulce de leche                               11
leche de arroz                               10
leche vegetal                                 9
leche semidesnatada                           7
leche de avena                                7
leche tibia                                   4
leche condensada 1 bote pequeño               3
virutas de chocolate con leche                2
leche entera                                  2
chocolate con leche de cobertura              2
chocolate con leche cobertura                 1
leche evaporada o nata                  

In [28]:
filter_mantequilla = recipe_df['mantequilla'] > 86
recipe_df[filter_mantequilla]["ingredient"].value_counts()

mantequilla                                    1796
mantequilla una nuez                              9
mantequilla fundida                               5
mantequilla ablandada                             5
mantequilla reblandecida                          4
mantequilla de cacahuete                          4
mantequilla nuez                                  4
mantequilla con sal                               3
mantequilla salada                                3
mantequilla a temperatura ambiente                2
mantequilla o margarina                           1
mantequilla light                                 1
mantequilla en pomada                             1
mantequilla 1 nuez                                1
mantequilla pomada                                1
mantequilla de cacahuete sin azúcar añadido       1
aceite de oliva suave o mantequilla               1
galletas de mantequilla                           1
mantequilla fría                                  1
mantequilla 

In [27]:
filter_harina = recipe_df['harina'] > 86
recipe_df[filter_harina]["ingredient"].value_counts()

harina                                           1659
harina de maíz                                    270
harina de trigo                                    80
harina de fuerza                                   61
harina integral                                    29
harina de maíz maicena                             22
harina de maíz (maicena)                           18
harina de almendra                                 18
harina de arroz                                    18
harina medido con el envase de yogur               11
harina de garbanzos                                10
harina de fuerza + 4 cucharadas                     5
harina de trigo sarraceno                           5
harina de repostería                                5
harina de tempura                                   4
harina de avena                                     4
harina de repostería sin gluten                     4
harina medidas con el envase de yogur               3
harina de trigo integral    

In [26]:
filter_sal = recipe_df['sal'] > 86
recipe_df[filter_sal]["ingredient"].value_counts()

sal                          3667
salsa mayonesa                102
salsa de soja                  93
salsa de tomate                72
salmón ahumado                 52
                             ... 
cacahuetes salados              1
salsa vinagreta                 1
salchichas tipo viena           1
sirope de salvia                1
salmón en cuatro supremas       1
Name: ingredient, Length: 121, dtype: int64

In [25]:
filter_azúcar = recipe_df['azúcar'] > 86
recipe_df[filter_azúcar]["ingredient"].value_counts()

azúcar                                    2099
azúcar glas                                861
azúcar moreno                              256
azúcar avainillado                          85
azúcar de coco                              16
                                          ... 
azúcar glas galletas de vainilla             1
perla de azúcar de color rosa y blanco       1
azúcar integral de caña                      1
azúcar perlado opcional                      1
azúcar + 2 cucharadas                        1
Name: ingredient, Length: 71, dtype: int64

In [39]:
filter_aceite = recipe_df['aceite'] > 86
recipe_df[filter_aceite]["ingredient"].value_counts()

aceite de oliva                                     2432
aceite                                               577
aceite de girasol                                    116
aceite de oliva virgen extra                          99
aceite de oliva suave                                 72
                                                    ... 
sardina en aceite de oliva                             1
bonito en aceite o en escabeche                        1
aceite de oliva de primera prensada en frío            1
ventresca de atún en conserva en aceite de oliva       1
aceite de oliva medida con el envase de yogur          1
Name: ingredient, Length: 68, dtype: int64

In [19]:
filter_zanahorias = recipe_df['zanahorias'] > 86
recipe_df[filter_zanahorias]["ingredient"].value_counts()

zanahoria                                                                           407
zanahorias                                                                          263
zanahorias baby                                                                       8
zanahorias rallada                                                                    4
zanahorias pequeñas                                                                   3
fondant en forma de zanahorias                                                        1
zanahorias medianas                                                                   1
zanahorias con las hojas                                                              1
calabacín sirve cualquier resto de verduras cocidas (acelgas puerros zanahorias)      1
zanahorias peladas y ralladas                                                         1
zanahorias mediana                                                                    1
conservas de verduras guisantes 

In [20]:
filter_queso = recipe_df['queso'] > 86
recipe_df[filter_queso]["ingredient"].value_counts()

queso blanco                        245
queso rallado                       135
queso parmesano rallado             105
queso mascarpone                     75
queso parmesano                      64
                                   ... 
queso parmesano o emmental            1
queso vegano                          1
queso de cabra rollitos               1
queso mozzarella una bola fresca      1
queso mozzarella rallada light        1
Name: ingredient, Length: 130, dtype: int64

In [21]:
filter_bacon = recipe_df['bacon'] > 86
recipe_df[filter_bacon]["ingredient"].value_counts()

Series([], Name: ingredient, dtype: int64)

In [22]:
filter_lechuga = recipe_df['lechuga'] > 86
recipe_df[filter_lechuga]["ingredient"].value_counts()

lechuga                                                 54
lechugas                                                33
cogollo de lechuga                                      18
lechuga unas hojas                                      15
lechuga variadas                                         9
lechuga brotes variados                                  9
lechugas brotes variados                                 8
lechugas variada                                         8
lechuga brotes                                           7
lechuga hoja de roble                                    6
cogollos de lechuga                                      6
lechugas brotes                                          5
lechuga hojas variadas                                   5
lechuga unas hojas variadas                              5
lechuga unos brotes                                      4
lechuga romana                                           4
lechugas hojas variadas                                 

In [33]:
filter_calabacín = recipe_df['calabacín'] > 80
recipe_df[filter_calabacín]["ingredient"].value_counts()

calabacín             261
calabacines            58
calabacín pequeño      12
calabacín grande        4
calabacín pequeños      3
calabacín medianos      3
calabacín mediano       3
calabacín troceado      1
calabacín bola          1
flor de calabacín       1
calabacín rallado       1
calabacín baby          1
Name: ingredient, dtype: int64

In [24]:
filter_patatas = recipe_df['patatas'] > 86
recipe_df[filter_patatas]["ingredient"].value_counts()

patatas                               211
patata                                200
patatas medianas                       53
patatas pequeñas                       24
patatas baby                            6
patatas grandes                         6
paté                                    5
patatas nuevas pequeñas                 2
patatas cocidas                         1
patatas nuevas                          1
patatas medianas o 4 pequeñas           1
patatas violeta                         1
patatas baby cocidas                    1
patatas tipo baby                       1
patatas chips                           1
patatas del tipo violeta                1
patatas medianas y alargadas            1
patatas pequeñas cocidas envasadas      1
patatas pequeñas rojas                  1
patatas pequeñas o 1 grande             1
Name: ingredient, dtype: int64

In [ ]:
def clean_ingredients(ingred_column):
    #ingred = ast.literal_eval(ingred_column)
    ingredients_list = []
    for ing in ingred_column:
        clean_ingred = re.sub(" para.*$| el peso de.$| medi[a-z][a-z]? con.*$", "", ing)
        ingredients_list.append(clean_ingred)
    return (ingredients_list)

recipe_df["ingredients_list"] = recipe_df.apply(lambda x: clean_ingredients(x['ingredients']), axis=1)